In [1]:
import random
import json
import nltk
import pickle
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/dai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
lemmantizer=WordNetLemmatizer()
data = open("intense_spanish.json").read()
intents=json.loads(data)#json.loads(open("intense.json"))

In [3]:
print(intents)

{'intents': [{'tag': 'saludos', 'patterns': ['hola', 'buenas', 'buenos dias', 'que tal?', '¿que tal?', 'klk', 'hi', 'hello'], 'responses': ['!hola¡', '!hola humano¡', 'buenas, ¿en qué puedo ayudarte?', 'hola, ¿en qué puedo ayudarte?', '¿en qué puedo ser útil?']}, {'tag': 'nombre', 'patterns': ['¿cómo te llamas?', '¿cuál es tu nombre?', '¿cómo puedo llamarte?', 'te llamas Robot?'], 'responses': ['me llamo FINANCIALYTICS-BOT, tu bot de apoyo en Financialitica', 'Llámame FINANCIALYTICS-BOT,soy bot de apoyo en Financialitica', 'soy tu bot de apoyo en Financialitica,FINANCIALYTICS-BOT']}, {'tag': 'edad', 'patterns': ['¿qué edad tienes?', '¿que edad tienes?', '¿cuál es tu edad?', 'edad?', 'los bots tienen edad?'], 'responses': ['mi creador tiene 34 años, yo tengo menos de 1', 'soy joven en edad humana, pero aprendo más rápido,tengo menos de 1 año']}]}


In [4]:
for intent in intents['intents']:
    print(intent['patterns'])

['hola', 'buenas', 'buenos dias', 'que tal?', '¿que tal?', 'klk', 'hi', 'hello']
['¿cómo te llamas?', '¿cuál es tu nombre?', '¿cómo puedo llamarte?', 'te llamas Robot?']
['¿qué edad tienes?', '¿que edad tienes?', '¿cuál es tu edad?', 'edad?', 'los bots tienen edad?']


In [5]:
words=[]
clases=[]
documents=[]
ignore_letters=["?","¿","!","¡",".","-","_",","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append(((word_list),intent['tag']))


        if intent['tag'] not in clases:
            clases.append(intent['tag'])

words = [lemmantizer.lemmatize(word) for word in words 
       if not word in ignore_letters]

words = sorted( set(words))


pickle.dump(words, open('words_spanish.pkl','wb'))
pickle.dump(clases, open('clases_spanish.pkl','wb'))

In [6]:
documents

[(['hola'], 'saludos'),
 (['buenas'], 'saludos'),
 (['buenos', 'dias'], 'saludos'),
 (['que', 'tal', '?'], 'saludos'),
 (['¿que', 'tal', '?'], 'saludos'),
 (['klk'], 'saludos'),
 (['hi'], 'saludos'),
 (['hello'], 'saludos'),
 (['¿cómo', 'te', 'llamas', '?'], 'nombre'),
 (['¿cuál', 'es', 'tu', 'nombre', '?'], 'nombre'),
 (['¿cómo', 'puedo', 'llamarte', '?'], 'nombre'),
 (['te', 'llamas', 'Robot', '?'], 'nombre'),
 (['¿qué', 'edad', 'tienes', '?'], 'edad'),
 (['¿que', 'edad', 'tienes', '?'], 'edad'),
 (['¿cuál', 'es', 'tu', 'edad', '?'], 'edad'),
 (['edad', '?'], 'edad'),
 (['los', 'bots', 'tienen', 'edad', '?'], 'edad')]

In [7]:
words

['Robot',
 'bot',
 'buenas',
 'buenos',
 'dia',
 'e',
 'edad',
 'hello',
 'hi',
 'hola',
 'klk',
 'llama',
 'llamarte',
 'los',
 'nombre',
 'puedo',
 'que',
 'tal',
 'te',
 'tienen',
 'tienes',
 'tu',
 '¿cuál',
 '¿cómo',
 '¿que',
 '¿qué']

In [8]:
clases

['saludos', 'nombre', 'edad']

In [9]:
#one hot encoding 


training=[]
output_empty=[0]*len(clases)

for document in documents:
    bag=[]
    word_patterns = document[0]
    word_patterns = [lemmantizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    

    output_row = list(output_empty) #making a copy of output_empty
    output_row[clases.index(document[1])] = 1
    training.append([bag,output_row])
random.shuffle(training)
training= np.array(training)


train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-9-4aee5438df8c>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training= np.array(training)


In [10]:
train_x[0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]

In [11]:
(np.array(train_y))

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [12]:
class Chatbot_trainer(tf.keras.Model):
    def __init__(self,num_classes=3,activation='relu',units=128,**kwargs):
        super().__init__(**kwargs)
        
        self.num_classes=num_classes
        # self.inputs=tf.keras.Input(shape=(26,))
        self.flat=tf.keras.layers.Flatten()        
        self.dense1=tf.keras.layers.Dense(units,activation=activation)
        self.dropout1=tf.keras.layers.Dropout(0.5)
        self.dense2=tf.keras.layers.Dense((units/2), activation=activation)
        self.dropout2=tf.keras.layers.Dropout(0.2)
        self.dense3=tf.keras.layers.Dense(self.num_classes, activation='softmax')
        



    def call(self,inputs):
        # input=self.input_base(input_shape=(len(input_length[0]),))
        # input1=self.inputs()
        x = self.flat(inputs)
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)
        output = self.dense3(x)

        return  output



        

In [13]:
model=Chatbot_trainer(num_classes=3)
model.compile(optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01,weight_decay=1e-6,momentum=0.90,nesterov=True),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])



In [14]:
hist=model.fit(np.array(train_x), np.array(train_y),
                 epochs=400, batch_size=2, verbose=1,shuffle=True)





# tf_train_y=tf.data.Dataset.from_tensor_slices(np.array(train_y))
# model.fit(tf_train_x,tf_train_y)

Epoch 1/400
9/9 [==============================] - 2s 4ms/step - loss: 1.0879 - accuracy: 0.3529 
Epoch 2/400
9/9 [==============================] - 0s 4ms/step - loss: 0.9757 - accuracy: 0.7059
Epoch 3/400
9/9 [==============================] - 0s 3ms/step - loss: 0.8829 - accuracy: 0.7647
Epoch 4/400
9/9 [==============================] - 0s 3ms/step - loss: 0.7988 - accuracy: 0.7647
Epoch 5/400
9/9 [==============================] - 0s 3ms/step - loss: 0.5887 - accuracy: 0.8824
Epoch 6/400
9/9 [==============================] - 0s 3ms/step - loss: 0.4235 - accuracy: 1.0000
Epoch 7/400
9/9 [==============================] - 0s 3ms/step - loss: 0.4168 - accuracy: 1.0000
Epoch 8/400
9/9 [==============================] - 0s 3ms/step - loss: 0.2144 - accuracy: 1.0000
Epoch 9/400
9/9 [==============================] - 0s 3ms/step - loss: 0.2055 - accuracy: 0.9412
Epoch 10/400
9/9 [==============================] - 0s 3ms/step - loss: 0.1233 - accuracy: 1.0000
Epoch 11/400
9/9 [==========

In [15]:
model.summary()

Model: "chatbot_trainer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  3456      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  8256      
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-t

In [20]:
model.evaluate(np.array(train_x)[len(train_x)-4:len(train_x)-1],np.array(train_y)[len(train_x)-4:len(train_x)-1])

1/1 [==============================] - 0s 20ms/step - loss: 1.8636e-05 - accuracy: 1.0000


[1.863601391960401e-05, 1.0]

In [21]:
# train_x[0:8]

In [18]:
# pip install pydot

In [19]:
# tf.keras.utils.plot_model(model)

In [23]:
model.save('Chatbot_base_spanish.tf',hist)

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(26, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5c80b90d0>, 140624880983952), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(26, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5c80b90d0>, 140624880983952), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5b87fa340>, 140624880688544), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5b87fa340>, 140624880688544), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5c805e790>, 140624880073200), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 3), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5c806db20>, 140624880395728), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5b87901f0>, 140624880395152), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fe5b87901f0>, 140624880395152), {}).


INFO:tensorflow:Assets written to: Chatbot_base_spanish.tf/assets


INFO:tensorflow:Assets written to: Chatbot_base_spanish.tf/assets


In [ ]:
# model.summary()

In [ ]:
# for i in tf.data.Dataset.from_tensor_slices(np.array(train_x)).take(1):
#     print (i.shape)

In [ ]:
np.array(train_y).shape[1]

In [ ]:
np.array(train_x).shape[0]

In [ ]:
(len(np.array(train_x)[0]),)

In [ ]:
len(train_y)

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Dense(128, input_shape=(len(train_x[0]), ),
#                 activation='relu'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(len(train_y[0]), 
#                 activation='softmax'))

In [ ]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd, metrics=['accuracy'])
# hist = model.fit(np.array(train_x), np.array(train_y),
#                  epochs=200, batch_size=5, verbose=1)